In [1]:
pip install python3-discogs-client

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import discogs_client
import time
import pandas as pd
import numpy as np

In [3]:
d = discogs_client.Client('Your arbitrary user string here', user_token='your user token here')

In [4]:
# 108475 is the artist id for Greg Phillinganes.
greg = d.artist(108475)
# All the releases/masters related to Greg Phillinganes.
greg_rel = greg.releases
# The release list can be paginated, 22 pages in total.
greg_rel_page0 = greg_rel.page(0)
greg_rel_page1 = greg_rel.page(1)
greg_rel_page2 = greg_rel.page(2)
greg_rel_page3 = greg_rel.page(3)
greg_rel_page4 = greg_rel.page(4)
greg_rel_page5 = greg_rel.page(5)

In [5]:
def copy_data(rel_entry):
    # We need the following line to force the program to copy all the data of the release.
    # Without it, the size of the resultant dict will be only 12. This is the most painful part.
    dum = rel_entry.url
    # Set the time value to 0.95 or larger, if received "too many requests" error from server.
    time.sleep(0.9)
    return rel_entry.data

# start_time = time.time()
# master_list = []
# # greg_rel is the whole list, with 21 pages.
# for cur_rel in greg_rel_page0:
#     version_list = []
#     # Each release's type is either master or release
#     if cur_rel.data["type"] == "master":
#         for rel_entry in cur_rel.versions:
#             version_list.append(copy_data(rel_entry))
#     else:
#         version_list.append(copy_data(cur_rel))
#     print(cur_rel.title)
#     master_list.append(version_list)
# end_time = time.time()
# print(end_time - start_time)

# # Each entry of the master_list is a version_list.
# # A version_list contains every version of a master release.
# # Each entry of the version_list is a dict containing everything of a version(release/album).
# # All versions -> version_list -> master_list.

In [6]:
start_time = time.time()
master_list_page1 = []
# greg_rel is the whole list, with 21 pages.
for cur_rel in greg_rel_page0:
    version_list = []
    # Each release's type is either master or release
    if cur_rel.data["type"] == "master":
        for rel_entry in cur_rel.versions:
            version_list.append(copy_data(rel_entry))
    else:
        version_list.append(copy_data(cur_rel))
    print(cur_rel.title)
    master_list_page1.append(version_list)
end_time = time.time()
print(end_time - start_time)


Girl Talk
Baby, I Do Love You
Takin' It Up All Night
Significant Gains
Pulse
Lazy Nina
Countdown To Love / Behind The Mask
Behind The Mask
Playin' With Fire
Love Fantasy / Baby, I Do Love You
Say Yes 
Heartbreak Hotel / Working Day & Night
Triumph
Heartbreak Hotel
Hotel Sentimental = Heartbreak Hotel
Streets Of Fire - Music From The Original Motion Picture Soundtrack
Private Joy
Weird Science - Music From The Motion Picture Soundtrack
Destiny / Triumph / Victory
Sweet And Lovely - The Best Of Nino Tempo & April Stevens
Disco Breaks Mastercuts
The Essential Jacksons
The Very Best Of The Jacksons
Jest
The Ultimate Collection
Possibilities
I Hear Angels Calling
Heat
The 75th Birthday Celebration
Triumph / Destiny
Sound Advice
Scream
Triumph
Lovely One
Headed For The Future
Headed For The Future
The Ultimate Collection
Possibilities
The Gap Band II
The Gap Band / The Gap Band II / The Gap Band III
Earthmover
K.Y. And The Curb / When I'm With You (Live Version)
Vibrations
Songs In The Key O

# Expected data

### Performer Metadata
Role, Category, Track Title, Track Artists,	Album, Distributing Label, Release Year, Duration, Instruments, 	Number of Featured Performers, UPC, Evidence(url), 


### Conditional Data
Version(remix, radio, edit...), Additional Credit(producer, arranger, conductor), Year of Recording	Country of Recording(blank if same release year), Country of Release, Percentage of FP share, 

### Optional Data
ISRC, Genre, Format, Catalog #, Notes

## Release Dataframe

```
Releases:
    -Release 1:
        -tracklist
            -track 1
                -extraartists
                    -greg
                -......
            -track 2
                -extraartists
                    -greg
                    -......
            -track 3
                -extraartists
                    -others
                    -......
            ...
        -id
        -year
        -...
```

* Convert master data from list of dict to dataframe. 
* Drop some useless columns. 
* Get artists name of each releases
* Get role of our client if there is such information. If there is no such information, leave the cell 'unknown'

## Note: 
1. Roles might be displayed in extraartists column of releases dataframe or hidden deeper in tracklist column. 
2. The roles in artists column are always empty, so we extract role information from the extraartists fields.  

In [7]:
release_df = pd.DataFrame()
for master in master_list_page1:
    release_df = pd.concat([release_df, pd.DataFrame(master)], ignore_index=True)
    
# drop some useless columns
release_df = release_df.drop(['videos', 'labels', 'status','stats', 'companies', 'format', 'community', 'images', 'artists_sort'], axis = 1)

# extract artists name and id from artists dictionary
release_df.artists = [[(i['name'], i['id']) if i != '' else i for i in d ] for d in release_df.artists]

# extract formats from formats dictionary
release_df.formats = [[i['name'] for i in d ]for d in release_df.formats ]

In [8]:
# get roles if there is such information from releases
cleaned_release_roles = []
for extraartists in release_df.extraartists:
    role = []
    if extraartists:
        for extraartist in extraartists:
            if extraartist['id'] == 108475:
                role.append(extraartist['role'])
    else:
        role.append('unknown')
    cleaned_release_roles.append(role)
release_df['release_role'] = cleaned_release_roles
release_df['release_role'] = [';'.join(map(str, l)) for l in release_df.release_role]

In [9]:
release_df.loc[release_df.id == 8944727]

,id,label,country,title,major_formats,catno,released,resource_url,thumb,year,...,styles,tracklist,extraartists,estimated_weight,blocked_from_sale,type,role,artist,trackinfo,release_role
26,8944727,RCA,Japan,Pulse,[CD],SICP 4857,2016-07-27,https://api.discogs.com/releases/8944727,https://i.discogs.com/iYt4tKgzX4oIz0JAuQ1L7P2T...,2016,...,"[RnB/Swing, Synth-pop, Disco, Soul]","[{'position': '1', 'type_': 'track', 'title': ...","[{'name': 'Toshikazu Kanazawa', 'anv': '金澤寿和',...",85.0,False,NaN,NaN,NaN,NaN,


## Track Dataframe  --> Extract credits from tracks

> The track data comes from the tracklist column in each release

> Basiclly, each release has a different number of tracks, and each track has different extra artists. The main idea is to keep all the tracks first and then drop the ones with extra artists that do not include our client (GP). 

> In the process, if the client has credits in certain tracks, we get the corresponding roles. If there are no additional artists, we keep the track as well.


In [10]:
# extract tracks from all releases
tracks_df = pd.DataFrame([dict(**{'release_id':rel_id}, **y) for rel_id, v in zip(release_df.id, release_df.tracklist.values) for y in v], )
tracks_df = tracks_df.replace(np.nan,'',regex=True)

In [11]:
# get roles if there is such information from tracks

cleaned_roles = []
for extraartists in tracks_df.extraartists:
    role = []
    if extraartists:
        for extraartist in extraartists:
            if extraartist['id'] == 108475:
                role.append(extraartist['role'])
    else:
        role.append('unknown')
    cleaned_roles.append(role)
tracks_df['track_role'] = cleaned_roles
tracks_df['track_role'] = [';'.join(map(str, l)) for l in tracks_df.track_role]
tracks_df = tracks_df.replace('',np.nan,regex=True)
tracks_df = tracks_df[tracks_df['track_role'].notna()]

In [12]:
tracks_df.extraartists = tracks_df.extraartists.replace(np.nan,'',regex=True)
tracks_df['track_extraartists'] = [[(i['name'], i['id']) if i != '' else i for i in d ] for d in tracks_df.extraartists]
tracks_df['track_extraartists'] = [';'.join(map(str, l)) for l in tracks_df.track_extraartists]

In [13]:
tracks_df.artists = tracks_df.artists.replace(np.nan,'',regex=True)
tracks_df['track_artists'] = [[(i['name'], i['id']) if i != '' else i for i in d ] for d in tracks_df.artists]
tracks_df['track_artists'] = [';'.join(map(str, l)) for l in tracks_df.track_artists]
tracks_df = tracks_df.replace('',np.nan,regex=True)
tracks_df = tracks_df.drop(['extraartists', 'artists'], axis = 1)

In [14]:
tracks_df

,release_id,position,type_,title,duration,sub_tracks,track_role,track_extraartists,track_artists
0,10245075,A,track,Girl Talk,3:43,NaN,unknown,NaN,NaN
1,10245075,B,track,Girl Talk,3:43,NaN,unknown,NaN,NaN
2,21792616,A,track,Girl Talk,3:43,NaN,unknown,NaN,NaN
3,21792616,B,track,Maxxed Out,5:02,NaN,unknown,NaN,NaN
4,3597653,A,track,"Baby, I Do Love You",3:52,NaN,unknown,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12856,3582264,B5,track,Music On My Mind,3:27,NaN,unknown,NaN,NaN
12857,11174682,1,track,Let Him In / Walk Softly / Dancin' To Keep Fro...,10:12,NaN,unknown,NaN,NaN
12858,11174682,2,track,Dancin' To Keep From Cryin' (cont.) / This One...,10:11,NaN,unknown,NaN,NaN
12859,11174682,3,track,"The Perfect Day / Delta Road / Glory, Glory",10:13,NaN,unknown,NaN,NaN


## Join release data into each track

> After we get all tracks, we can combine the release data to each track based on the same release id


## Note: 
1. There are some overlapped data in the dataframe, for instance, tracklist, role, artists, extraartists. 
2. If both track_role and release_role are 'unknown' which means there is no role information found in original data.

In [15]:
release_df = release_df.rename(columns={"id": "release_id"})
release_df = release_df.drop(['extraartists'], axis=1)
release_df = release_df.drop(['tracklist'], axis=1)

In [16]:
result_df = tracks_df.join(release_df.set_index('release_id'), on='release_id', lsuffix='_track', rsuffix='_release')

In [17]:
idx = np.unique( result_df.index.values, return_index = True )[1]
result_df = result_df.iloc[idx]

In [18]:
result_df

,release_id,position,type_,title_track,duration,sub_tracks,track_role,track_extraartists,track_artists,label,...,identifiers,genres,styles,estimated_weight,blocked_from_sale,type,role,artist,trackinfo,release_role
0,10245075,A,track,Girl Talk,3:43,NaN,unknown,NaN,NaN,Planet (15),...,"[{'type': 'Rights Society', 'value': 'ASCAP'}]","[Electronic, Funk / Soul]","[Rhythm & Blues, Soul, Synth-pop]",60.0,False,NaN,NaN,NaN,NaN,Producer
1,10245075,B,track,Girl Talk,3:43,NaN,unknown,NaN,NaN,Planet (15),...,"[{'type': 'Rights Society', 'value': 'ASCAP'}]","[Electronic, Funk / Soul]","[Rhythm & Blues, Soul, Synth-pop]",60.0,False,NaN,NaN,NaN,NaN,Producer
2,21792616,A,track,Girl Talk,3:43,NaN,unknown,NaN,NaN,Planet (15),...,[],[Rock],NaN,60.0,False,NaN,NaN,NaN,NaN,unknown
3,21792616,B,track,Maxxed Out,5:02,NaN,unknown,NaN,NaN,Planet (15),...,[],[Rock],NaN,60.0,False,NaN,NaN,NaN,NaN,unknown
4,3597653,A,track,"Baby, I Do Love You",3:52,NaN,unknown,NaN,NaN,Planet (15),...,[],"[Electronic, Funk / Soul]","[Rhythm & Blues, Soul, Synth-pop]",230.0,False,NaN,NaN,NaN,NaN,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12856,3582264,B5,track,Music On My Mind,3:27,NaN,unknown,NaN,NaN,Blue Note,...,[],[Funk / Soul],"[Soul, Disco]",230.0,False,NaN,NaN,NaN,NaN,Keyboards
12857,11174682,1,track,Let Him In / Walk Softly / Dancin' To Keep Fro...,10:12,NaN,unknown,NaN,NaN,Blue Note,...,[],[Funk / Soul],"[Soul, Disco]",85.0,False,NaN,NaN,NaN,NaN,Keyboards
12858,11174682,2,track,Dancin' To Keep From Cryin' (cont.) / This One...,10:11,NaN,unknown,NaN,NaN,Blue Note,...,[],[Funk / Soul],"[Soul, Disco]",85.0,False,NaN,NaN,NaN,NaN,Keyboards
12859,11174682,3,track,"The Perfect Day / Delta Road / Glory, Glory",10:13,NaN,unknown,NaN,NaN,Blue Note,...,[],[Funk / Soul],"[Soul, Disco]",85.0,False,NaN,NaN,NaN,NaN,Keyboards


In [19]:
# convert dataframe to csv file
import os  
os.makedirs('output', exist_ok=True)
result_df.to_csv('output/v3_sample_output_Greg_Phillinganes.csv') 

In [20]:
# convert dataframe to xlsx file
# pip install openpyxl
result_df.to_excel("output/v3_sample_output_Greg_Phillinganes.xlsx")